# Autocombat TOP and StrokeMRI harmonized datasets

Note this must be run in the `comscan6` environment

## import libraries

In [ ]:
import os
import sys

import pandas as pd
import numpy as np

sys.path.insert(0, '../../') # path to functions

import cvasl.harmony as har
import cvasl.vendor.comscan.neurocombat as autocombat

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
# Datasets for this work

MRI_path = '../our_datasets/StrokeMRI/'
TOP_path = '../our_datasets/TOP/'
file_name = 'TrainingDataComplete.csv'

TOP_file = os.path.join(TOP_path, file_name)
MRI_file = os.path.join(MRI_path, file_name)

TOP = pd.read_csv(TOP_file, index_col=0)
MRI = pd.read_csv(MRI_file, index_col=0)

In [ ]:
TOP = TOP.drop([ 'ID'], axis= 1)
TOP['Site'] = 0
TOP.head(3)

In [ ]:
MRI = MRI.drop(['ID'],axis = 1)
MRI.tail(5)

In [ ]:
TOP.head(3)

In [ ]:
TOPMRI = pd.concat([TOP, MRI])
TOPMRI.head(3) 

In [ ]:
TOPMRI.columns = TOPMRI.columns.str.lower()

In [ ]:
TOPMRI['decade']=(TOPMRI['age']/10).round()
#TOPMRI['decade']

In [ ]:
TOPMRI = TOPMRI.sort_values(by='age')
TOPMRI.reset_index(inplace=True)
TOPMRI['fine_grain'] = TOPMRI['age'].rolling(2).sum()/2
#TOPMRI

In [ ]:
TOPMRI[2:].fine_grain.iloc[::2] = TOPMRI[:].fine_grain.iloc[1::2]
#TOPMRI['fine_grain']

In [ ]:
TOPMRI['fine_grain'][0] = TOPMRI['fine_grain'][1]
TOPMRI['fine_grain']

In [ ]:
TOPMRI['decade'].unique()

In [ ]:
TOPMRI.columns

In [ ]:
combat = autocombat.Combat(
    features=[ 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',],
    sites=[ "site"], discrete_covariates=['sex'],continuous_covariates=['decade'],)

fg_combat = autocombat.Combat(
    features=[ 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',],
    sites=[ "site"], discrete_covariates=['sex'],continuous_covariates=['fine_grain'],)


In [ ]:
print(combat.fit(TOPMRI))
print(fg_combat.fit(TOPMRI))

In [ ]:
print(combat.gamma_star_)
print(fg_combat.gamma_star_)

In [ ]:
transformed_TOPMRI= combat.transform(TOPMRI)
transformed_TOPMRI.head(3)

In [ ]:
fg_transformed_TOPMRI= fg_combat.transform(TOPMRI)
fg_transformed_TOPMRI.head(3)

In [ ]:
TOPMRI.head(3)

In [ ]:
transformed_TOPMRI= combat.transform(TOPMRI)
transformed_TOPMRI.head(3)

In [ ]:
fg_transformed_TOPMRI= fg_combat.transform(TOPMRI)
fg_transformed_TOPMRI.head(3)

In [ ]:
transformed_TOPMRI.columns

In [ ]:
TOP_transformed = transformed_TOPMRI[ transformed_TOPMRI['site']==0]
TOP_transformed = TOP_transformed.drop(['site', 'decade','fine_grain', 'index'], axis=1)
TOP_transformed.head(3)

In [ ]:
fg_TOP_transformed = fg_transformed_TOPMRI[ fg_transformed_TOPMRI['site']==0]
fg_TOP_transformed = fg_TOP_transformed.drop(['site', 'decade','fine_grain', 'index'], axis=1)
fg_TOP_transformed.head(3)

In [ ]:
MRI_transformed = transformed_TOPMRI[ transformed_TOPMRI['site']==1]
MRI_transformed = MRI_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
MRI_transformed.head(3)

In [ ]:
fg_MRI_transformed = fg_transformed_TOPMRI[fg_transformed_TOPMRI['site']==1]
fg_MRI_transformed = fg_MRI_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
fg_MRI_transformed.head(3)

In [ ]:
TOP_transformed.to_csv('harm_results/autocombat/autocom_harm_top1.csv')
MRI_transformed.to_csv('harm_results/autocombat/autocom_harm_mri1.csv')

fg_TOP_transformed.to_csv('harm_results/autocombat/fg_autocom_harm_top1.csv')
fg_MRI_transformed.to_csv('harm_results/autocombat/fg_autocom_harm_mri1.csv')

# Data quality check

In [ ]:
#TOP

In [ ]:
#neuro_harm_top

In [ ]:
TOP_transformed.isna().sum().sum()

In [ ]:
MRI_transformed.isna().sum().sum()

In [ ]:
number_columns = ['sex', 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',]

In [ ]:
#neuro_harm_top.head(3)
(TOP_transformed[number_columns] < 0).sum()

In [ ]:
(MRI_transformed[number_columns] < 0).sum()

In [ ]:
MRI_transformed

## So this new variable (WMHvol_WMvol ) is again problematic

 We will wait about logging it until we see other columns we may want to log across all datasets